In [1]:
!pip install -q openai langchain tiktoken faiss-cpu

In [4]:
import os
from openai import OpenAI

key = ""
os.environ['OPENAI_API_KEY'] = key

client = OpenAI()

In [5]:
import pandas as pd

df = pd.read_csv("qa_dataset.csv", sep="|")
df.head()

,category,question,answer,embedding
0,예금 · 적금,듀얼K 입출금통장의 '남길금액'과 '플러스박스'는 어떤 점이 다른가요?,듀얼K입출금통장 (남길금액)\n여유자금을 남길금액으로 설정하고 한달 동안 잔액을 유...,"[-0.018170368, -0.102951586, 1.0141064, -0.871..."
1,예금 · 적금,듀얼K 입출금통장의 '남길금액'과 '플러스박스'를 동시에 이용할 수 있나요?,듀얼K 입출금통장을 보유하신 고객은 남길금액과 플러스박스(파킹통장)를 동시에 이용할...,"[-0.107609905, -0.22749503, 1.3711764, -0.7877..."
2,예금 · 적금,입출금 통장별로 플러스박스를 만들 수 있나요?,네 가능합니다. 플러스박스는 고객님이 보유하신 입출금통장과 연결하여 1인 최대 10...,"[-0.090952665, -0.29497996, 1.2406822, -0.5032..."
3,예금 · 적금,듀얼K 입출금통장에 플러스박스를 연결할 수 있나요?,네 가능합니다.\n고객님이 보유하신 당행 입출금통장에 플러스박스를 연결할 수 있습니...,"[-0.2557224, -0.1423537, 1.4077793, -0.6369744..."
4,예금 · 적금,입출금통장 잔액이 부족하면 플러스박스 잔액에서 자동으로 출금 되나요?,아니요. 자동으로 출금되지 않습니다.\n플러스박스 보관금액은 연결된 입출금통장 잔액...,"[-0.22084534, 0.11942373, 1.186573, -0.1382266..."


In [6]:
client = OpenAI()

def get_embedding(text, model="text-embedding-ada-002"):
    text = text.replace("\n", " ")
    return client.embeddings.create(input = [text], model=model).data[0].embedding

df['ada_embedding'] = df['question'].apply(lambda x: get_embedding(x, model='text-embedding-ada-002'))

In [7]:
df.to_csv("results.csv", sep="|", index=False)

In [8]:
df.head()

,category,question,answer,embedding,ada_embedding
0,예금 · 적금,듀얼K 입출금통장의 '남길금액'과 '플러스박스'는 어떤 점이 다른가요?,듀얼K입출금통장 (남길금액)\n여유자금을 남길금액으로 설정하고 한달 동안 잔액을 유...,"[-0.018170368, -0.102951586, 1.0141064, -0.871...","[-0.0023889727890491486, 0.0060088736936450005..."
1,예금 · 적금,듀얼K 입출금통장의 '남길금액'과 '플러스박스'를 동시에 이용할 수 있나요?,듀얼K 입출금통장을 보유하신 고객은 남길금액과 플러스박스(파킹통장)를 동시에 이용할...,"[-0.107609905, -0.22749503, 1.3711764, -0.7877...","[-0.013954252935945988, -0.00400847801938653, ..."
2,예금 · 적금,입출금 통장별로 플러스박스를 만들 수 있나요?,네 가능합니다. 플러스박스는 고객님이 보유하신 입출금통장과 연결하여 1인 최대 10...,"[-0.090952665, -0.29497996, 1.2406822, -0.5032...","[-0.013343286700546741, -0.014095021411776543,..."
3,예금 · 적금,듀얼K 입출금통장에 플러스박스를 연결할 수 있나요?,네 가능합니다.\n고객님이 보유하신 당행 입출금통장에 플러스박스를 연결할 수 있습니...,"[-0.2557224, -0.1423537, 1.4077793, -0.6369744...","[-0.01965808868408203, 0.003127714851871133, 0..."
4,예금 · 적금,입출금통장 잔액이 부족하면 플러스박스 잔액에서 자동으로 출금 되나요?,아니요. 자동으로 출금되지 않습니다.\n플러스박스 보관금액은 연결된 입출금통장 잔액...,"[-0.22084534, 0.11942373, 1.186573, -0.1382266...","[-0.011873620562255383, -0.024180201813578606,..."


In [9]:
import faiss
import numpy as np

embeddings = np.array(df['ada_embedding'].tolist())
dim = embeddings.shape[1]

In [10]:
ids = [i for i in range(len(embeddings))]
index = faiss.IndexFlatL2(dim)
index = faiss.IndexIDMap2(index)
index.add_with_ids(embeddings, ids)

In [11]:
distnaces, indices = index.search(embeddings[0].reshape(1,-1),3)

print(distnaces)
print(indices)

[[0.         0.07966946 0.13640139]]
[[0 1 3]]


In [12]:
embeddings[0].reshape(1,-1).shape

(1, 1536)

In [13]:
q = "플러스박스 연결하는 방법 알려줘"
q_embedding = get_embedding(q)

In [14]:
q_search = np.array(q_embedding).reshape(1,-1)
distnaces, indices = index.search(q_search,3)

print(distnaces)
print(indices)

[[0.28690213 0.34518945 0.3497604 ]]
[[3 2 6]]


In [15]:
index_nm = "test.index"

faiss.write_index(index, index_nm)
index = faiss.read_index(index_nm)

In [16]:
# https://wikidocs.net/book/14314
# teddy langchain

In [17]:
%pip install -U langchain-community faiss-cpu langchain-openai tiktoken

Note: you may need to restart the kernel to use updated packages.


In [18]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import Chroma, AtlasDB, FAISS
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

In [19]:
import csv
csv.field_size_limit(100000000)
csv.field_size_limit()

100000000

In [20]:
loader = CSVLoader(
    file_path="fsi_smart_faq_ko.csv",
    source_column="Source",
    encoding="utf-8"
)
context_documents = loader.load()

In [21]:
print(len(context_documents))

context_documents[0:3]

89


[Document(page_content='no: 89\nCategory: 타기관OTP 이용등록방법 알려주세요\nInformation: 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다. \n[경로]\n- 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록  \n- 신한 쏠(SOL) 로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→   OTP이용등록\n \n ※ OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.\n\n기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.\ntype: 인터넷뱅킹\nSource: 신한은행', metadata={'source': '신한은행', 'row': 0}),
 Document(page_content='no: 88\nCategory: 공동인증서와 금융인증서 차이점이 무엇인가요?\nInformation: 공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자거래범용(수수료 4,400원) 인증서가 있으며 유효기간은 1년입니다. \n공동인증서는 하드디스크나 이동식디스크, 휴대폰 등 원하시는 기기에 저장해서 이용할 수 있습니다.\n인증서를 저장한 매체에서는 인증서 비밀번호로 편리하게 이용할 수 있으나 다른 기기에서 이용하려면 기기마다 복사하거나 이동식디스크에 저장해서 휴대해야 하는 불편함이 있을 수 있습니다.\n\n금융인증서는 금융인증서는 금융결제원의 클라우드에 저장하여 이용하는 인증서로 발급/이용 시에 클라우드에 접속이 필요합니다.\n금융결제원 클라우드에 연결만 가능하다면 어디서든 편리하게 이용 가능하지만, PC나 USB, 휴대폰 등 다른 기기로 복사는 불가합니다.(유효기간 3년/발급 수수료 무료)\n※ 클라우드 계정 연결을 위해 휴대폰을 통한 ARS, SMS, 마이인포앱 인증 절차가 필요합니다.\ntype: 인증서\nSource: 신한은행', metadata={'source': '신한은행', 'row': 1}),

In [23]:
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings

openai_embeddings = OpenAIEmbeddings()

index_creator = VectorstoreIndexCreator(
    vectorstore_cls=FAISS,
    embedding=openai_embeddings,
    #text_splitter=CharacterTextSplitter(chunk_size=700, chunk_overlap=0),
    #text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=200), 
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=200)
)

index = index_creator.from_loaders([loader])

/Users/databiz/miniconda3/envs/llm/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [24]:
print(len(context_documents))

context_documents[0:3]

89


[Document(page_content='no: 89\nCategory: 타기관OTP 이용등록방법 알려주세요\nInformation: 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다. \n[경로]\n- 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록  \n- 신한 쏠(SOL) 로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→   OTP이용등록\n \n ※ OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.\n\n기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.\ntype: 인터넷뱅킹\nSource: 신한은행', metadata={'source': '신한은행', 'row': 0}),
 Document(page_content='no: 88\nCategory: 공동인증서와 금융인증서 차이점이 무엇인가요?\nInformation: 공동인증서 (구 공인인증서)는 용도에 따라 은행/신용카드/보험용 무료 인증서와 전자거래범용(수수료 4,400원) 인증서가 있으며 유효기간은 1년입니다. \n공동인증서는 하드디스크나 이동식디스크, 휴대폰 등 원하시는 기기에 저장해서 이용할 수 있습니다.\n인증서를 저장한 매체에서는 인증서 비밀번호로 편리하게 이용할 수 있으나 다른 기기에서 이용하려면 기기마다 복사하거나 이동식디스크에 저장해서 휴대해야 하는 불편함이 있을 수 있습니다.\n\n금융인증서는 금융인증서는 금융결제원의 클라우드에 저장하여 이용하는 인증서로 발급/이용 시에 클라우드에 접속이 필요합니다.\n금융결제원 클라우드에 연결만 가능하다면 어디서든 편리하게 이용 가능하지만, PC나 USB, 휴대폰 등 다른 기기로 복사는 불가합니다.(유효기간 3년/발급 수수료 무료)\n※ 클라우드 계정 연결을 위해 휴대폰을 통한 ARS, SMS, 마이인포앱 인증 절차가 필요합니다.\ntype: 인증서\nSource: 신한은행', metadata={'source': '신한은행', 'row': 1}),

In [25]:
# Save
index.vectorstore.save_local("fsi_faq_indexer_ko")

In [26]:
# Read
index_ = FAISS.load_local("fsi_faq_indexer_ko", openai_embeddings, allow_dangerous_deserialization=True)

In [27]:
def filter_and_remove_score(res, cutoff_score = 200, variance=1.3):
    # Get the lowest score
    lowest_score = min(score for doc, score in res)
    print('lowest_score : ', lowest_score)
    # If the lowest score is over 200, return an empty list
    if lowest_score > cutoff_score:
        return []
    # Calculate the upper bound for scores
    upper_bound = lowest_score * variance

    
    # Filter the list and remove the score
    res = [doc for doc, score in res if score <= upper_bound]

    return res


def get_search_answer (query, bank="", k=1):
    search_query = query
    similar_docs = get_similiar_docs(search_query, k=k, bank=bank)
    llm_query = '검색된 내용을 no. 별로 요약한 문장으로 알려줘.'
    if not similar_docs:
        llm_query = query

    llm_answer = chain.run(input_documents=similar_docs, question=llm_query)
    return llm_answer

def get_similiar_docs(query, k=1, fetch_k=50, score=True, bank=""):

    #print (query)
    
    if score:         
        similar_docs = index_.similarity_search_with_score(
            query,
            k=k,
            fetch_k=fetch_k,
            filter=dict(source=bank)
        )
        similar_docs=filter_and_remove_score(similar_docs)
    else:        
        similar_docs = index_.similarity_search(
            query,
            k=k,
            fetch_k=fetch_k,
            filter=dict(source=bank)
        )
        
    return similar_docs

In [28]:
def get_answer(query, bank="", k=1):
                
    search_query = query
    
    similar_docs = get_similiar_docs(search_query, k=k, bank=bank)
    
    print("similar_docs : ", similar_docs)
    llm_query = ''+query+' Category에 대한 Information을 찾아서 설명해주세요.'
    
    if not similar_docs:
        llm_query = query

    answer = chain.run(input_documents=similar_docs, question=llm_query)
    
    return answer

In [29]:
llm = ChatOpenAI()
seperator = "||SPEPERATOR||"

prompt_template = ''.join(["{context}", seperator, "{question}"])
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain = load_qa_chain(llm=llm, chain_type="stuff", prompt=PROMPT, verbose=True)

chain.run(input_documents='', question='안녕하세요')

/Users/databiz/miniconda3/envs/llm/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/Users/databiz/miniconda3/envs/llm/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
||SPEPERATOR||안녕하세요

> Finished chain.

> Finished chain.


'안녕하세요! 무엇을 도와드릴까요?'

In [30]:
%%time
question = "타기관OTP 등록 방법 알려주세요"
response = get_answer(question, bank="신한은행", k=3)

print (f'question: {question}')
print('==========================')
print('\n')
print (f'response: \n {response}')

lowest_score :  0.25762394
similar_docs :  [Document(page_content='no: 89\nCategory: 타기관OTP 이용등록방법 알려주세요\nInformation: 타기관에서 발급받으신 OTP가 통합OTP카드인 경우 당행에 등록하여 이용가능합니다. \n[경로]\n- 인터넷뱅킹 로그인→ 사용자관리→인터넷뱅킹관리→OTP이용등록  \n- 신한 쏠(SOL) 로그인→ 전체메뉴→설정/인증→ 이용중인 보안매체선택→   OTP이용등록\n \n ※ OTP이용등록후 재로그인을 하셔야 새로 등록된 보안매체가 적용됩니다.\n\n기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.\ntype: 인터넷뱅킹\nSource: 신한은행', metadata={'source': '신한은행', 'row': 0}), Document(page_content='no: 80\nCategory: 모바일 OTP 사용하고 있는데 인터넷뱅킹에서는 어떻게 이용하는건가요?\nInformation: 인터넷뱅킹 거래시 보안매체 입력단계에 [인증요청] 버튼 클릭하면 핸드폰으로 푸시메세지가 전달됩니다. \n푸시메세지 터치하여 모바일 OTP 비밀번호 6자리 입력, 인증처리 완료후 인터넷뱅킹에서 이후의 절차를 진행하시면 됩니다.\n※ 스마트폰에 모바일 OTP 푸시(PUSH) 메세지가 오지 않는 경우 메뉴 직접 실행 \n- 신한 쏠(SOL) [로그인 하지 않음]> 전체메뉴> 설정/인증> 보안매체 전체보기> 모바일OTP> 인증\n기타 궁금하신 내용은 신한은행 고객센터 1599-8000로 문의하여 주시기 바랍니다.\ntype: 인터넷뱅킹\nSource: 신한은행', metadata={'source': '신한은행', 'row': 9})]


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
no: 89
Category